In [6]:
import torch
import torch.nn as nn
from fastai.vision.all import *
from fastai.data.all import *
from fastai.distributed import *
import pandas as pd
from pathlib import Path
import time
from modules_th.video_block import * 
from modules_th.inflator import *
from modules_th.triplet_loss import *

In [7]:
class NullModel(Module):
    def __init__(self): pass
    def forward(*x, **kwargs): return x

class NullLoss(Module):
    def __init__(self): pass
    def forward(*x, **kwargs): return torch.tensor(0.,requires_grad=True)
    
def null_splitter(m): return [[],[]]

In [8]:
def read_data(eugenio):
    path_data = '/mnt/data/eugeniomarinelli/videodata/df.csv' if eugenio else '/mnt/data/adrianlopez/Videos/Charades/df.csv'
    df = pd.read_csv(path_data, index_col=0).dropna()
    return df 

def get_vid_path(df:pd.Series): return Video(df['paths'].split('\n'))
def get_lbls(df:pd.Series): return df['lbl']

path_data = Path('/mnt/data/adrianlopez/Videos/Charades/df.csv')
df = read_data(False)

In [9]:
tfms = [[get_vid_path, ResizeTime(l=20), Video.create],
        [get_lbls, Categorize()]]

splits = RandomSplitter(valid_pct=0.2, seed=42)(df)
n_lbl, n_el = 4, 8
splits = [uniformize_dataset(idxs, df.iloc[idxs]['lbl'] , n_lbl=n_lbl, n_el=n_el) for idxs in splits]

dsets = Datasets(df, tfms, splits=splits)

In [10]:
for i in range(-1,-10,-1): print(dsets.valid[i][1])

TensorCategory(139)
TensorCategory(139)
TensorCategory(139)
TensorCategory(139)
TensorCategory(139)
TensorCategory(139)
TensorCategory(139)
TensorCategory(139)
TensorCategory(93)


In [11]:
dls = dsets.dataloaders(bs=n_el*n_lbl,
                        shuffle_train=False,
                        after_item=[Resize(224), ToTensor()],
                        after_batch=[IntToFloatTensor(), Normalize.from_stats(*imagenet_stats)])

In [12]:
dls.one_batch()

(TensorVideo([[[[[0.4000, 0.3922, 0.3922,  ..., 0.0000, 0.0000, 0.0000],
            [0.4039, 0.3961, 0.3922,  ..., 0.0000, 0.0000, 0.0000],
            [0.4157, 0.4078, 0.3961,  ..., 0.0000, 0.0000, 0.0000],
            ...,
            [0.2510, 0.2706, 0.2863,  ..., 0.0000, 0.0000, 0.0000],
            [0.2510, 0.2706, 0.2902,  ..., 0.0000, 0.0000, 0.0000],
            [0.2549, 0.2745, 0.2980,  ..., 0.0000, 0.0000, 0.0000]],
 
           [[0.4039, 0.3961, 0.3882,  ..., 0.0000, 0.0000, 0.0000],
            [0.4078, 0.4000, 0.3922,  ..., 0.0000, 0.0000, 0.0000],
            [0.4196, 0.4118, 0.4000,  ..., 0.0000, 0.0000, 0.0000],
            ...,
            [0.2627, 0.2784, 0.2941,  ..., 0.0000, 0.0000, 0.0000],
            [0.2549, 0.2667, 0.2902,  ..., 0.0000, 0.0000, 0.0000],
            [0.2510, 0.2667, 0.2902,  ..., 0.0000, 0.0000, 0.0000]],
 
           [[0.4039, 0.3961, 0.3882,  ..., 0.0000, 0.0000, 0.0000],
            [0.4078, 0.4000, 0.3922,  ..., 0.0000, 0.0000, 0.0000],
   

In [13]:
head, body = inflate(create_head(1024, len(dls.vocab), lin_ftrs=[256])), inflate(create_body(resnet34, cut=-2)) 
learn = Learner(dls, 
#                 NullModel(),
                TLModel(body,head),
#                 loss_func=NullLoss(),
                MixedLoss(alpha=1.0, margin=.2),
#                 splitter=null_splitter,
                splitter=my_splitter,
#                 opt=Optimizer([0], noop),
#                 metrics=lambda *x: 0)
                metrics=tl_accuracy)

In [14]:
learn.fine_tune(1)

KeyboardInterrupt: 

In [1]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_triplet_loss.ipynb.
Converted 02_inflator.ipynb.
Converted 03_video_block.ipynb.
Converted 04_datasets.ipynb.
Converted 05_first_inflated_NN.ipynb.
Converted index.ipynb.


In [ ]:
# default_exp first_inflated_NN